## VOICE ALERT

In [5]:
import numpy as np
import cv2
import time
from tensorflow.keras.models import load_model
import pyttsx3

# Set the size of the input images
img_size = (224, 224)

# Load models
yawn_model = load_model('yawn_detection_model.keras')
drowsiness_model = load_model('drowsiness_detection_model.keras')

# Load Haar cascades
face_cascade = cv2.CascadeClassifier(r"C:\Users\HP\OneDrive\FINAL DROWSY\Code\Requirement files\haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(r"C:\Users\HP\OneDrive\FINAL DROWSY\Code\Requirement files\haarcascade_eye.xml")

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize variables
prev_yawn_status = False
yawn_start_time = None
total_yawns = 0
yawn_threshold = 5
alert_played = False

closed_eyes_start_time = None
DROWSINESS_THRESHOLD_SECONDS = 3
closed_eyes_duration = 0  

# Initialize TTS engine
engine = pyttsx3.init()

def play_alert_sound(message):
    engine.say(message)
    engine.runAndWait()

def update_yawn_status(yawn_status):
    global prev_yawn_status, yawn_start_time, total_yawns, alert_played
    if yawn_status != prev_yawn_status:
        if yawn_status:
            yawn_start_time = time.time()
        else:
            if yawn_start_time is not None:
                yawn_duration = time.time() - yawn_start_time
                if yawn_duration > 1:
                    total_yawns += 1
                    if total_yawns >= yawn_threshold and not alert_played:
                        play_alert_sound("Warning! Drowsiness detected due to multiple yawns.")
                        alert_played = True
                        total_yawns = 0
        prev_yawn_status = yawn_status

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame for yawn detection
        roi_rgb = cv2.resize(frame, img_size)
        roi_rgb = roi_rgb.astype(np.float32) / 255.0
        roi_rgb = np.expand_dims(roi_rgb, axis=0)

        # Predict yawning status
        yawn_prediction = yawn_model.predict(roi_rgb)
        yawn_status = yawn_prediction[0][0] > 0.5
        update_yawn_status(yawn_status)

        # Convert the frame to grayscale for drowsiness detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        eye_status = "Awake"
        elapsed_time = 0

        for (fx, fy, fw, fh) in faces:
            roi_gray = gray[fy:fy + fh, fx:fx + fw]
            eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))

            for (ex, ey, ew, eh) in eyes:
                eye_roi_gray = roi_gray[ey:ey + eh, ex:ex + ew]
                eye_roi_gray = cv2.equalizeHist(eye_roi_gray)
                eye_roi_gray = cv2.GaussianBlur(eye_roi_gray, (5, 5), 0)
                final_eye_image = cv2.resize(eye_roi_gray, (224, 224))
                final_eye_image_rgb = cv2.cvtColor(final_eye_image, cv2.COLOR_GRAY2RGB)
                final_eye_image_rgb = final_eye_image_rgb.astype(np.float32) / 255.0
                final_eye_image_rgb = np.expand_dims(final_eye_image_rgb, axis=0)

                drowsiness_prediction = drowsiness_model.predict(final_eye_image_rgb)
                drowsiness_status = "Awake" if drowsiness_prediction[0][0] > 0.5 else "Drowsy"

                if drowsiness_status == "Drowsy":
                    if closed_eyes_start_time is None:
                        closed_eyes_start_time = time.monotonic()
                        alert_played = False
                    else:
                        elapsed_time = time.monotonic() - closed_eyes_start_time
                        if elapsed_time >= DROWSINESS_THRESHOLD_SECONDS:
                            eye_status = "Drowsy"
                            if not alert_played:
                                play_alert_sound("Warning! Drowsy due to prolonged eye closure.")
                                alert_played = True
                        closed_eyes_duration = elapsed_time  # Update the closed eyes duration
                else:
                    if closed_eyes_start_time is not None:
                        closed_eyes_duration = time.monotonic() - closed_eyes_start_time
                    closed_eyes_start_time = None
                    alert_played = False

        # Display the status on the frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        if eye_status == "Drowsy":
            display_text = f"Asleep for {elapsed_time:.2f} seconds"
            color = (0, 0, 255)  # Red
        elif closed_eyes_start_time:
            elapsed_time = time.monotonic() - closed_eyes_start_time
            display_text = f"Drowsy for {elapsed_time:.2f} seconds"
            color = (0, 255, 255)  # Yellow
        elif closed_eyes_duration > 0:
            display_text = f"Asleep for {closed_eyes_duration:.2f} seconds"
            color = (0, 255, 0)  # Green
        else:
            display_text = "Awake"
            color = (0, 255, 0)  # Green

        cv2.putText(frame, display_text, (20, 40), font, 1, color, 2, cv2.LINE_AA)
        cv2.putText(frame, f'Yawns Count: {total_yawns}', (20, 100), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow('Drowsiness and Yawn Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━

In [ ]:
import matplotlib.pyplot as plt

# Data for Eye State Dataset
eye_labels = ['Public Data ', 'Custom Data']
eye_sizes = [80, 20]
eye_colors = ['#99ccff','#66b3ff']

# Data for Mouth State Dataset
mouth_labels = ['Public Data', 'Custom Data ']
mouth_sizes = [75, 25]
mouth_colors = ['#ccccff','#9a9aff']

# Create subplots for both pie charts
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Eye State Dataset Pie Chart
ax1.pie(eye_sizes, labels=eye_labels, colors=eye_colors, autopct='%1.1f%%', startangle=90, explode=(0, 0.1))
ax1.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
ax1.set_title('Eye State Dataset')

# Mouth State Dataset Pie Chart
ax2.pie(mouth_sizes, labels=mouth_labels, colors=mouth_colors, autopct='%1.1f%%', startangle=90, explode=(0, 0.1))
ax2.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
ax2.set_title('Mouth State Dataset')

# Show the pie charts
plt.tight_layout()
plt.show()
